In [1]:
import itertools
import functools
import numpy as np

In [2]:
def combinations(x, degree):
    return itertools.combinations_with_replacement(x, degree)

In [3]:
def compute_new_features(items):
    return functools.reduce(lambda x, y: x * y, items)

In [4]:
{each:compute_new_features(each) for each in combinations([2,3], 2)} #running a test

{(2, 2): 4, (2, 3): 6, (3, 3): 9}

In [5]:
[compute_new_features(each) for each in combinations(np.array([2,3]), 2)] #running a test

[4, 6, 9]

In [6]:
def polynomial_transform(x, degree):
    if x.ndim == 1:
        x = x[:, None]

    features = [np.ones(len(x))] #a list of np-arrays with all 1.

    print(features)
    for degree in range(1, degree + 1):
        for item in combinations(x.T, degree):
            print(item)
            features.append(compute_new_features(item))

    print(features)
            
    return np.asarray(features).T

In [7]:
#feature vector [2,3] gets converted into two examples, first with 2 and second with 3 weights[[2],[3]]
polynomial_transform(np.array([2, 3]), 2) # Want combinations of 2.

[array([1., 1.])]
(array([2, 3]),)
(array([2, 3]), array([2, 3]))
[array([1., 1.]), array([2, 3]), array([4, 9])]


array([[1., 2., 4.],
       [1., 3., 9.]])

In [8]:
#Here, in a single example, we've 2 and 3 as two weights.  Want combinations of 2.
polynomial_transform(np.array([[2, 3]]), 2)

[array([1.])]
(array([2]),)
(array([3]),)
(array([2]), array([2]))
(array([2]), array([3]))
(array([3]), array([3]))
[array([1.]), array([2]), array([3]), array([4]), array([6]), array([9])]


array([[1., 2., 3., 4., 6., 9.]])

In [9]:
#Here, in a single example, we've two examples, each with two weights.  Want combinations of 2.
polynomial_transform(np.array([[2, 3],[4, 5]]), 2)

[array([1., 1.])]
(array([2, 4]),)
(array([3, 5]),)
(array([2, 4]), array([2, 4]))
(array([2, 4]), array([3, 5]))
(array([3, 5]), array([3, 5]))
[array([1., 1.]), array([2, 4]), array([3, 5]), array([ 4, 16]), array([ 6, 20]), array([ 9, 25])]


array([[ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [10]:
#Here, in a single example, we've two examples, each with two weights.  Want combinations of 3.
polynomial_transform(np.array([[2, 3],[4, 5]]), 3)

[array([1., 1.])]
(array([2, 4]),)
(array([3, 5]),)
(array([2, 4]), array([2, 4]))
(array([2, 4]), array([3, 5]))
(array([3, 5]), array([3, 5]))
(array([2, 4]), array([2, 4]), array([2, 4]))
(array([2, 4]), array([2, 4]), array([3, 5]))
(array([2, 4]), array([3, 5]), array([3, 5]))
(array([3, 5]), array([3, 5]), array([3, 5]))
[array([1., 1.]), array([2, 4]), array([3, 5]), array([ 4, 16]), array([ 6, 20]), array([ 9, 25]), array([ 8, 64]), array([12, 80]), array([ 18, 100]), array([ 27, 125])]


array([[  1.,   2.,   3.,   4.,   6.,   9.,   8.,  12.,  18.,  27.],
       [  1.,   4.,   5.,  16.,  20.,  25.,  64.,  80., 100., 125.]])

In [11]:
#Here, in a single example, we've two examples, each with two weights.  Want combinations of 3.
polynomial_transform(np.array([[2, 3],[4, 5],[6,7]]), 3)

[array([1., 1., 1.])]
(array([2, 4, 6]),)
(array([3, 5, 7]),)
(array([2, 4, 6]), array([2, 4, 6]))
(array([2, 4, 6]), array([3, 5, 7]))
(array([3, 5, 7]), array([3, 5, 7]))
(array([2, 4, 6]), array([2, 4, 6]), array([2, 4, 6]))
(array([2, 4, 6]), array([2, 4, 6]), array([3, 5, 7]))
(array([2, 4, 6]), array([3, 5, 7]), array([3, 5, 7]))
(array([3, 5, 7]), array([3, 5, 7]), array([3, 5, 7]))
[array([1., 1., 1.]), array([2, 4, 6]), array([3, 5, 7]), array([ 4, 16, 36]), array([ 6, 20, 42]), array([ 9, 25, 49]), array([  8,  64, 216]), array([ 12,  80, 252]), array([ 18, 100, 294]), array([ 27, 125, 343])]


array([[  1.,   2.,   3.,   4.,   6.,   9.,   8.,  12.,  18.,  27.],
       [  1.,   4.,   5.,  16.,  20.,  25.,  64.,  80., 100., 125.],
       [  1.,   6.,   7.,  36.,  42.,  49., 216., 252., 294., 343.]])